In [2]:
import numpy as np
import pandas as pd

from methods.model_fitting_utilities import softmax_neg_log_likelihood
from scipy import stats
from scipy.optimize import minimize

In [3]:
summary = pd.read_csv('./data/model_fitting_outputs/summary_data.csv', sep=';')
final_judgements = pd.read_csv('./data/model_fitting_outputs/final_judgements.csv')
summary['information_gained'] = (summary['prior_entropy'] - summary['posterior_entropy']) / summary['prior_entropy']

In [4]:

print(f'Final judgements shape: {final_judgements.shape}')
posteriors = {}

for model_name in summary.model_name.unique():
    posteriors[model_name] = pd.read_csv(f'./data/model_fitting_outputs/{model_name}/posteriors.csv')
    print(f'{model_name} posteriors shape: {posteriors[model_name].shape}')

Final judgements shape: (15625, 1092)
normative posteriors shape: (15625, 1092)
LC_discrete posteriors shape: (15625, 1092)
LC_discrete_attention posteriors shape: (15625, 1092)
change_discrete posteriors shape: (15625, 1092)


In [5]:
print('Datapoints per experiment:')
print('Total:', f'N={summary.pid.nunique()}', f'length={summary.shape[0]}')
print('Experiment 2:', f'N={summary[summary.experiment == "experiment_1"].pid.nunique()}', f'length={summary[summary.experiment == "experiment_1"].shape[0]}')
print('Experiment 2:', f'N={summary[summary.experiment == "experiment_2"].pid.nunique()}', f'length={summary[summary.experiment == "experiment_2"].shape[0]}')
print('Experiment 3:', f'N={summary[summary.experiment == "experiment_3"].pid.nunique()}', f'length={summary[summary.experiment == "experiment_3"].shape[0]}')

# Split data into 3 dataset
summary_1 = summary[summary.experiment == "experiment_1"]
summary_2 = summary[summary.experiment == "experiment_2"]
summary_3 = summary[summary.experiment == "experiment_3"]


Datapoints per experiment:
Total: N=302 length=4344
Experiment 2: N=60 length=480
Experiment 2: N=121 length=1936
Experiment 3: N=121 length=1928


In [10]:
summary_3

,utid,pid,experiment,difficulty,scenario,model_name,ground_truth,posterior_map,posterior_judgement,prior_judgement,prior_entropy,posterior_entropy,model_specs,information_gained
0,3_566feba6b937e400052d33b2_finance_congruent,566feba6b937e400052d33b2,experiment_3,congruent,finance,normative,[ 1. 1. -0.5 -0.5 1. 1. ],[ 1. 1. -0.5 -0.5 1. 1. ],[ 0. 1. -0.5 -0.5 1. 1. ],[ 1. 1. -0.5 -0.5 1. 1. ],9.390294,7.981602e-36,NaN,1.000000
1,3_566feba6b937e400052d33b2_finance_congruent,566feba6b937e400052d33b2,experiment_3,congruent,finance,LC_discrete,[ 1. 1. -0.5 -0.5 1. 1. ],[1. 0.5 0. 0. 0.5 1. ],[ 0. 1. -0.5 -0.5 1. 1. ],[ 1. 1. -0.5 -0.5 1. 1. ],9.390294,6.192261e-05,NaN,0.999993
2,3_566feba6b937e400052d33b2_finance_congruent,566feba6b937e400052d33b2,experiment_3,congruent,finance,LC_discrete_attention,[ 1. 1. -0.5 -0.5 1. 1. ],[ 1. 1. -0.5 -0.5 1. 1. ],[ 0. 1. -0.5 -0.5 1. 1. ],[ 1. 1. -0.5 -0.5 1. 1. ],9.390294,9.390294e+00,NaN,0.000000
3,3_566feba6b937e400052d33b2_dampened_generic,566feba6b937e400052d33b2,experiment_3,generic,dampened,normative,[-1. 0.5 0. 1. 0. 0. ],[-1. 0.5 0. 1. 0. 0. ],[0.5 0. 0.5 0. 0. 0. ],NaN,13.931326,9.939385e-01,NaN,0.928654
4,3_566feba6b937e400052d33b2_dampened_generic,566feba6b937e400052d33b2,experiment_3,generic,dampened,LC_discrete,[-1. 0.5 0. 1. 0. 0. ],[-1. -1. 0. 1. 0. 0.],[0.5 0. 0.5 0. 0. 0. ],NaN,13.931326,6.574633e-03,NaN,0.999528
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4339,3_61717173748006894b2b54ff_neg_chain_generic_2,61717173748006894b2b54ff,experiment_3,generic_2,neg_chain,change_discrete,[-1 0 0 -1 0 0],[-0.5 -1. 0. -0.5 0.5 -0.5],[-0.5 1. 0. -0.5 0. 0. ],NaN,13.931559,1.301234e+00,NaN,0.906598
4340,3_6176966806de000024ed0ddf_crime_congruent,6176966806de000024ed0ddf,experiment_3,congruent,crime,change_discrete,[-1. 0.5 0.5 1. 0. 1. ],[ 1. 1. 1. 1. 0. -1.],[ 1. 1. 1. 0. 0. -1.],[-1. 0.5 0.5 1. 0. 1. ],13.728335,4.208897e-01,NaN,0.969342
4341,3_6176966806de000024ed0ddf_dampened_generic,6176966806de000024ed0ddf,experiment_3,generic,dampened,change_discrete,[-1. -0.5 0. -1. 0. 0. ],[ 0. -1. 0.5 -1. 1. -1. ],[ 0. 0. -0.5 -0.5 0. 0.5],NaN,13.931559,2.997061e-04,NaN,0.999978
4342,3_6176966806de000024ed0ddf_finance_incongruent,6176966806de000024ed0ddf,experiment_3,incongruent,finance,change_discrete,[ 0. -0.5 0.5 -1. 1. 0. ],[ 0. -1. 1. -0.5 1. 0. ],[ 0.5 -1. -0.5 -0.5 1. 1. ],[ 0.5 1. 0. 0. -0.5 -1. ],13.753367,3.559470e-03,NaN,0.999741


In [6]:
congruent = summary_3[summary_3.difficulty == 'congruent']
incongruent = summary_3[summary_3.difficulty == 'incongruent']

congruent_norm = congruent[congruent.model_name == 'normative'].information_gained
incongruent_norm = incongruent[incongruent.model_name == 'normative'].information_gained

congruent_lc = congruent[congruent.model_name == 'LC_discrete'].information_gained
incongruent_lc = incongruent[incongruent.model_name == 'LC_discrete'].information_gained

congruent_lc_a = congruent[congruent.model_name == 'LC_discrete_attention'].information_gained
incongruent_lc_a = incongruent[incongruent.model_name == 'LC_discrete_attention'].information_gained

congruent_change = congruent[congruent.model_name == 'change_discrete'].information_gained
incongruent_change = incongruent[incongruent.model_name == 'change_discrete'].information_gained

print('Information gained')
print('Normative')
print(f'Congruent: mean={congruent_norm.mean().round(4)}, std={congruent_norm.std().round(4)}')
print(f'Incongruent: mean={incongruent_norm.mean().round(4)}, std={incongruent_norm.std().round(4)}')
print(stats.ttest_ind(congruent_norm, incongruent_norm))

print('LC_discrete')
print(f'Congruent: mean={congruent_lc.mean().round(4)}, std={congruent_lc.std().round(4)}')
print(f'Incongruent: mean={incongruent_lc.mean().round(4)}, std={incongruent_lc.std().round(4)}')
print(stats.ttest_ind(congruent_norm, incongruent_norm))

print('LC_discrete_attention')
print(f'Congruent: mean={congruent_lc_a.mean().round(4)}, std={congruent_lc_a.std().round(4)}')
print(f'Incongruent: mean={incongruent_lc_a.mean().round(4)}, std={incongruent_lc_a.std().round(4)}')
print(stats.ttest_ind(congruent_lc_a, incongruent_lc_a))

print('Discrete Change')
print(f'Congruent: mean={congruent_change.mean().round(4)}, std={congruent_change.std().round(4)}')
print(f'Incongruent: mean={incongruent_change.mean().round(4)}, std={incongruent_change.std().round(4)}')
print(stats.ttest_ind(congruent_change, incongruent_change))

Information gained
Normative
Congruent: mean=0.9963, std=0.0243
Incongruent: mean=0.992, std=0.0316
Ttest_indResult(statistic=1.188732138968508, pvalue=0.2357298236523205)
LC_discrete
Congruent: mean=0.9953, std=0.0215
Incongruent: mean=0.9915, std=0.025
Ttest_indResult(statistic=1.188732138968508, pvalue=0.2357298236523205)
LC_discrete_attention
Congruent: mean=0.8817, std=0.27
Incongruent: mean=0.9117, std=0.2214
Ttest_indResult(statistic=-0.9414486846448392, pvalue=0.347429561901356)
Discrete Change
Congruent: mean=0.8789, std=0.2665
Incongruent: mean=0.911, std=0.211
Ttest_indResult(statistic=-1.0329441239542303, pvalue=0.30267852362438635)


In [7]:
# Select only data not links

judgements_arr = final_judgements[final_judgements.columns[6:]].to_numpy()

# Recover posteriors for each modes
posteriors_arr = {}
for model_name in summary.model_name.unique():
    posteriors_arr[model_name] = posteriors[model_name][posteriors[model_name].columns[6:]].to_numpy()


In [8]:
discretenans = np.argwhere(np.isnan(posteriors_arr['LC_discrete']))
temp=0

nanidx = np.unique(discretenans[:, 1])

model_names = ['normative', 'LC_discrete', 'LC_discrete_attention', 'change_discrete']
nLL = np.zeros(len(model_names))
optim_results = [None for i in model_names]
for i, model in enumerate(model_names):
    selection = np.delete(judgements_arr, nanidx, axis=1)
    dataset = np.delete(posteriors_arr[model], nanidx, axis=1)
    #nLL[i] = softmax_neg_log_likelihood(temp, dataset, judgements_arr)
    optim_results[i] = minimize(softmax_neg_log_likelihood, 
                              0, 
                              args=(dataset, selection))

In [16]:
print('Uniform negative log likelihood (baseline) = 10487.1 \n')
for i, model in enumerate(model_names):
    print(f'Model name: {model}')
    print(f'Negative Log Likelihood = {optim_results[i].fun}')
    print(f'Temperature = {optim_results[i].x[0]}')
    print(f'Optim. result: {optim_results[i].message} \n')

Uniform negative log likelihood (baseline) = 10487.1 

Model name: normative
Negative Log Likelihood = 10446.786038761678
Temperature = 4.493196817730923
Optim. result: Optimization terminated successfully. 

Model name: LC_discrete
Negative Log Likelihood = 10462.94930681456
Temperature = 3.387430856637707
Optim. result: Optimization terminated successfully. 

Model name: LC_discrete_attention
Negative Log Likelihood = 10457.622909812366
Temperature = 4.119688694925813
Optim. result: Optimization terminated successfully. 

Model name: change_discrete
Negative Log Likelihood = 10159.79672505864
Temperature = 7.107513697001946
Optim. result: Optimization terminated successfully. 

